In [ ]:
import requests
from bs4 import BeautifulSoup as bSoup
import re
import pandas as pd

In [ ]:
def listToString(l):
    str1 = ''
    for i in l:
        str1 += i
    return str1

In [ ]:
df = pd.read_csv('Gallary-Awards-gov2019.csv')
df.rename(columns = {'_ - name': 'name', '_ - external_url': 'url'}, inplace = True)
df['award_year']=''
df['service_num']=''

In [ ]:
df.head()

In [ ]:
for index in df.index:
    url = df['url'][index]
    page = requests.get(url)
    pageHTML = bSoup(page.content, 'html.parser')
    #converted entire HTML code to string as there is not specific HTML class assigned for year of award
    pageString = str(pageHTML)
    yearLine = re.findall("<div class=\"col-xs-12 col-sm-6 col-md-6 lside\"\>Year of Award<\/div>\\n<div class=\"col-xs-12 col-sm-6 col-md-6\"\>.*<\/div>", pageString)
    yearLine = listToString(yearLine)
    #this step was not directly performed as there might be other four digit numbers on a page
    awardYear = re.findall("\d{4}", yearLine)
    if len(awardYear) == 0:
        awardYear = 0
    df.loc[index, 'award_year'] = awardYear
    serviceLine = re.findall("<div class=\"col-xs-12 col-sm-6 col-md-6 lside\"\>Service No.<\/div>\\n<div class=\"col-xs-12 col-sm-6 col-md-6\"\>.*<\/div>", pageString)
    serviceLine = listToString(serviceLine)
    #there is not consitent pattern for service number, so eliminated the known pattern of HTML tags around it
    serviceNum = serviceLine.replace('<div class="col-xs-12 col-sm-6 col-md-6 lside">Service No.</div>\n<div class="col-xs-12 col-sm-6 col-md-6">', '').replace('</div>', '')
    if len(serviceNum) == 0:
        serviceNum = 'NOT FOUND'
    df.loc[index, 'service_num'] = serviceNum

In [ ]:
df.to_csv('GA2019Updated-IN.csv')